In [1]:
from sqlalchemy import create_engine
import sys
import os
import pandas as pd

In [4]:
if (os.environ.get("DB_TYPE") == None):
    from dotenv import load_dotenv
    from config.definitions import ROOT_DIR
    load_dotenv(os.path.join(ROOT_DIR, 'notebooks', 'config', 'conf', '.env'))

**Database Credentials**

In [5]:
db_type = os.environ.get("DB_TYPE")
user = os.environ.get('DB_USER')
pswd = os.environ.get('DB_PASS')
host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')

In [6]:
engine = create_engine(f"{db_type}+psycopg2://{user}:{pswd}@{host}/{db_name}")

In [10]:
con = engine.connect()

**Find Table Schemas in the Database**

In [11]:
con.execute("SELECT table_schema FROM information_schema.tables\
    WHERE table_schema != 'pg_catalog'\
    AND table_schema not in  ('information_schema', 'public')\
        AND table_type='BASE TABLE'").fetchall()

[('pa004',), ('pa004',), ('pa004',)]

**Name of the Tables**

In [13]:
query = "SELECT table_name FROM information_schema.tables\
        WHERE table_schema = 'pa004'"
exec = con.execute(query)

for table in exec.fetchall():
    print(table)

('users',)
('vehicle',)
('insurance',)


**Extracting the data from the tables**

In [14]:
users_query = 'SELECT * FROM pa004.users'
vehicle_query = 'SELECT * FROM pa004.vehicle'
insurance_query = 'SELECT * FROM pa004.insurance'

users = pd.read_sql_query(users_query, con=con)
insurance = pd.read_sql_query(insurance_query, con=con)
vehicle = pd.read_sql_query(vehicle_query, con=con)

In [15]:
users.head()

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


In [16]:
insurance.head()

,id,previously_insured,annual_premium,vintage,response
0,1,0,40454.0,217,1
1,2,0,33536.0,183,0
2,3,0,38294.0,27,1
3,4,1,28619.0,203,0
4,5,1,27496.0,39,0


In [17]:
vehicle.head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


**Joining Tables and Saving to `.csv` file**

In [57]:
raw_data = pd.merge(users, insurance, how='left', on='id')
raw_data = pd.merge(raw_data, vehicle, how='left', on='id')
raw_data.to_csv('data/raw_data.csv', index=False)

In [58]:
raw_data

,id,gender,age,region_code,policy_sales_channel,previously_insured,annual_premium,vintage,response,driving_license,vehicle_age,vehicle_damage
0,1,Male,44,28.0,26.0,0,40454.0,217,1,1,> 2 Years,Yes
1,2,Male,76,3.0,26.0,0,33536.0,183,0,1,1-2 Year,No
2,3,Male,47,28.0,26.0,0,38294.0,27,1,1,> 2 Years,Yes
3,4,Male,21,11.0,152.0,1,28619.0,203,0,1,< 1 Year,No
4,5,Female,29,41.0,152.0,1,27496.0,39,0,1,< 1 Year,No
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,26.0,26.0,1,30170.0,88,0,1,1-2 Year,No
381105,381106,Male,30,37.0,152.0,1,40016.0,131,0,1,< 1 Year,No
381106,381107,Male,21,30.0,160.0,1,35118.0,161,0,1,< 1 Year,No
381107,381108,Female,68,14.0,124.0,0,44617.0,74,0,1,> 2 Years,Yes


In [60]:
# Closing connection
con.close()

Exception during reset or similar
Traceback (most recent call last):
  File "e:\users\b_tc\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "e:\users\b_tc\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "e:\users\b_tc\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\engine\default.py", line 682, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

